In [2]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
import re

In [4]:
data = pd.read_csv('data.csv')
data

,text,l1,l2,l3,wiki_name,word_count
0,The 1994 Mindoro earthquake occurred on Novemb...,Event,NaturalEvent,Earthquake,1994_Mindoro_earthquake,59
1,The 1917 Bali earthquake occurred at 06:50 loc...,Event,NaturalEvent,Earthquake,1917_Bali_earthquake,68
2,The 1941 Colima earthquake occurred on April 1...,Event,NaturalEvent,Earthquake,1941_Colima_earthquake,194
3,The 1983 Coalinga earthquake occurred on May 2...,Event,NaturalEvent,Earthquake,1983_Coalinga_earthquake,98
4,The 2013 Bushehr earthquake occurred with a mo...,Event,NaturalEvent,Earthquake,2013_Bushehr_earthquake,61
...,...,...,...,...,...,...
342776,WCSH is the NBC-affiliated television station ...,Agent,Broadcaster,TelevisionStation,WCSH,198
342777,Al Jazeera America (AJAM) was an American basi...,Agent,Broadcaster,TelevisionStation,Al_Jazeera_America,226
342778,"CJOH-DT, VHF channel 13, is a CTV owned-and-op...",Agent,Broadcaster,TelevisionStation,CJOH-DT,234
342779,NTTV (North Texas Television) is a student tel...,Agent,Broadcaster,TelevisionStation,NTTV,126


In [6]:
def preprocess(text):
    text_input = re.sub('[^a-zA-Z1-9]+', ' ', str(text))
    output = re.sub(r'\d+', '',text_input)
    return output.lower().strip()

data['text'] = data.text.map(preprocess)
data

,text,l1,l2,l3,wiki_name,word_count
0,the mindoro earthquake occurred on november ...,Event,NaturalEvent,Earthquake,1994_Mindoro_earthquake,59
1,the bali earthquake occurred at local time ...,Event,NaturalEvent,Earthquake,1917_Bali_earthquake,68
2,the colima earthquake occurred on april at ...,Event,NaturalEvent,Earthquake,1941_Colima_earthquake,194
3,the coalinga earthquake occurred on may at e...,Event,NaturalEvent,Earthquake,1983_Coalinga_earthquake,98
4,the bushehr earthquake occurred with a momen...,Event,NaturalEvent,Earthquake,2013_Bushehr_earthquake,61
...,...,...,...,...,...,...
342776,wcsh is the nbc affiliated television station ...,Agent,Broadcaster,TelevisionStation,WCSH,198
342777,al jazeera america ajam was an american basic ...,Agent,Broadcaster,TelevisionStation,Al_Jazeera_America,226
342778,cjoh dt vhf channel is a ctv owned and operat...,Agent,Broadcaster,TelevisionStation,CJOH-DT,234
342779,nttv north texas television is a student telev...,Agent,Broadcaster,TelevisionStation,NTTV,126


In [7]:
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

data['text'] = data.text.map(remove_stopwords)
data

,text,l1,l2,l3,wiki_name,word_count
0,mindoro earthquake occurred november local tim...,Event,NaturalEvent,Earthquake,1994_Mindoro_earthquake,59
1,bali earthquake occurred local time january ja...,Event,NaturalEvent,Earthquake,1917_Bali_earthquake,68
2,colima earthquake occurred april utc local tim...,Event,NaturalEvent,Earthquake,1941_Colima_earthquake,194
3,coalinga earthquake occurred may exactly utc c...,Event,NaturalEvent,Earthquake,1983_Coalinga_earthquake,98
4,bushehr earthquake occurred moment magnitude a...,Event,NaturalEvent,Earthquake,2013_Bushehr_earthquake,61
...,...,...,...,...,...,...
342776,wcsh nbc affiliated television station souther...,Agent,Broadcaster,TelevisionStation,WCSH,198
342777,al jazeera america ajam american basic cable s...,Agent,Broadcaster,TelevisionStation,Al_Jazeera_America,226
342778,cjoh dt vhf channel ctv owned operated televis...,Agent,Broadcaster,TelevisionStation,CJOH-DT,234
342779,nttv north texas television student television...,Agent,Broadcaster,TelevisionStation,NTTV,126


In [8]:
def build_corpus(data):
    corpus = []
    for sentence in data.iteritems():
        word_list = sentence[1].split(" ")
        corpus.append(word_list)
    return corpus

corpus = build_corpus(data['text'])

In [9]:
model = Word2Vec(corpus, vector_size=100, min_count=1)

In [11]:
from gensim.models import KeyedVectors

# fit a 2D PCA model to the vectors
vectors = model.wv.get_normed_vectors()
words = list(model.wv.index_to_key)

words
pca = PCA(n_components=2)
PCA_result = pca.fit_transform(vectors)

In [12]:
model.wv.save_word2vec_format('word2vec_embedding.bin', binary=True)
